### 1. Title : Diabetes Diagnosis by Classification

### 2. Introduction : 
Diabetes is a chronic ailment affecting millions of individuals in the United States and represents a significant public health concern worldwide. The disease is characterized by elevated blood sugar levels and a markedly lower quality of life. The prevalence of diabetes in the US has been steadily increasing, and approximately 11% of the population suffers from diabetes today. Here, we are using the Diabetes Health Indicators Dataset collected in 2015, containing data collected through the Behavioural Risk Factor Surveillance System, a telephone survey relating to health outcomes. Using this dataset, we aim to investigate if we can accurately predict the presence or absence of diabetes based on these survey response data.

### 3. Preliminary exploratory data analysis :


In [11]:
.libPaths()

[1] "/opt/anaconda3/envs/myR/lib/R/library"

In [7]:
getwd("/opt/anaconda3/envs/myR/lib/R/library/tidyverse")

ERROR: Error in getwd("/opt/anaconda3/envs/myR/lib/R/library/tidyverse"): 사용되지 않은 인자 ("/opt/anaconda3/envs/myR/lib/R/library/tidyverse")


In [10]:
.libPaths('/opt/anaconda3/envs/myR/lib/R/library')

In [9]:
remove('/private/var/folders/1w/94w03vcn7rv_gpwvy1gmzjhw0000gn/T/RtmpWXGoBk/downloaded_packages')

Warning message in remove("/private/var/folders/1w/94w03vcn7rv_gpwvy1gmzjhw0000gn/T/RtmpWXGoBk/downloaded_packages"):
“객체 '/private/var/folders/1w/94w03vcn7rv_gpwvy1gmzjhw0000gn/T/RtmpWXGoBk/downloaded_packages'를 찾을 수 없습니다”


In [12]:
# # Installing Packages
install.packages("tidyverse")
# install.packages("RColorBrewer")
# update.packages("tidyverse")
# remove.packages("tidyverse")

# Loading in Libraries
library(tidyverse)
library(RColorBrewer)

# Loading in Data
url <- 'https://github.com/MatildaBae/dsci-100-2023W1-group45/raw/main/diabetes_binary_5050split_health_indicators_BRFSS2015.csv'
diab_data <- read_csv(url)


# Inspecting data
dim(diab_data)
str(diab_data)

‘systemfonts’, ‘textshaping’, ‘vroom’, ‘tzdb’, ‘haven’, ‘ragg’, ‘readr’(들)을 또한 설치합니다.


Warning message in install.packages("tidyverse"):
“패키지 ‘systemfonts’의 설치가 0이 아닌 종료상태를 가졌습니다”
Warning message in install.packages("tidyverse"):
“패키지 ‘tzdb’의 설치가 0이 아닌 종료상태를 가졌습니다”
Warning message in install.packages("tidyverse"):
“패키지 ‘textshaping’의 설치가 0이 아닌 종료상태를 가졌습니다”
Warning message in install.packages("tidyverse"):
“패키지 ‘vroom’의 설치가 0이 아닌 종료상태를 가졌습니다”
Warning message in install.packages("tidyverse"):
“패키지 ‘ragg’의 설치가 0이 아닌 종료상태를 가졌습니다”
Warning message in install.packages("tidyverse"):
“패키지 ‘readr’의 설치가 0이 아닌 종료상태를 가졌습니다”
Warning message in install.packages("tidyverse"):
“패키지 ‘haven’의 설치가 0이 아닌 종료상태를 가졌습니다”
Warning message in install.packages("tidyverse"):
“패키지 ‘tidyverse’의 설치가 0이 아닌 종료상태를 가졌습니다”
'.Library'내 HTML 패키지 목록을 업데이트 하고 있습니다

Making 'packages.html' ...
 완료



ERROR: Error in library(tidyverse): ‘tidyverse’이라고 불리는 패키지가 없습니다


In [ ]:
# Formatting column classes appropriately
age_levels <- c('[18,24]', 
                '[25,29]', 
                '[30,34]', 
                '[35,39]', 
                '[40,44]', 
                '[45,49]', 
                '[50,54]', 
                '[55,59]', 
                '[60,64]', 
                '[65,69]', 
                '[70,74]', 
                '[75,79]', 
                '[79,∞)')

diab_data <- diab_data %>% 
        mutate(across(everything(), as_factor)) %>%
        mutate(Diabetes_binary = fct_recode(Diabetes_binary, 'Case' = '1', 'Control' = '0')) %>%
        mutate(Diabetes_binary = factor(Diabetes_binary, levels = c('Case', 'Control'))) %>%
        mutate(Sex = fct_recode(Sex, 'Female' = '0', 'Male' = '1')) %>%
        mutate(Sex = factor(Sex, levels = c('Male', 'Female'))) %>%
        mutate(Age = fct_recode(Age, 
                                '[18,24]' = '1',
                                '[25,29]' = '2',
                                '[30,34]' = '3',
                                '[35,39]' = '4',
                                '[40,44]' = '5',
                                '[45,49]' = '6',
                                '[50,54]' = '7',
                                '[55,59]' = '8',
                                '[60,64]' = '9',
                                '[65,69]' = '10',
                                '[70,74]' = '11',
                                '[75,79]' = '12',
                                '[79,∞)' = '13')) %>%
        mutate(Age = factor(Age, levels = age_levels)) %>%
        mutate(BMI = as.numeric(BMI),
               MentHlth = as.numeric(MentHlth),
               GenHlth = as.numeric(GenHlth))


# Making sure that classes are balanced
diab_data %>% count(Diabetes_binary)

In [ ]:
# Exploring BMI data
summary_bmi <- diab_data %>% 
        select(Diabetes_binary, BMI) %>%
        group_by(Diabetes_binary) %>%
        summarize(mean_BMI = mean(BMI),
                  sd_BMI = sd(BMI))
summary_bmi

mean_bmi_case <- summary_bmi %>% 
        filter(Diabetes_binary == 'Case') %>% 
        pull(mean_BMI)
mean_bmi_control <- summary_bmi %>% 
        filter(Diabetes_binary == 'Control') %>% 
        pull(mean_BMI)



In [ ]:
# Plotting BMI frequency
bmi_plot <- diab_data %>%
        ggplot(aes(x = BMI, fill = Diabetes_binary)) +
        geom_histogram(bins = 30, 
                       color = 'white',
                       size = 0.5,
                       alpha = 0.5,
                       position = 'identity') +
        geom_vline(xintercept = mean_bmi_case, color = 'darkgreen', linewidth = 1) +
        geom_vline(xintercept = mean_bmi_control, color = 'red', linewidth = 1) +
        scale_x_continuous(limits = c(0, 60),
                           breaks = seq(0, 60, 10)) +
        labs(x = 'Body Mass Index (BMI)', 
             y = 'Count',
             fill = '') +
        ggtitle('BMI distribution across cases-controls') +
        scale_fill_brewer(palette = 'Set2') +
        theme_classic() +
        theme(text = element_text(size = 12))

bmi_plot

In [ ]:
# General and Mental Health scores
diab_data %>% select(Diabetes_binary, GenHlth, MentHlth) %>%
        group_by(Diabetes_binary) %>%
        summarize(across(GenHlth:MentHlth, mean, .names = 'mean_{.col}'),
                  across(GenHlth:MentHlth, sd, .names = 'sd_{.col}'))


In [ ]:
# Population plot
diab_data %>% 
        count(Diabetes_binary, 
              Age, 
              Sex) %>%
        mutate(n = ifelse(Sex == 'Male', 
                          n * -1, 
                          n)) %>%
        ggplot(aes(x = Age, 
                   y = n, 
                   fill = Diabetes_binary)) +
        geom_bar(stat = 'identity', 
                 position = 'dodge',
                 color = 'white',
                 width = 1) +
        labs(x = 'Age bracket', 
             y = 'Count',
             fill = '') +
        ggtitle('Age and gender distribution across case-controls') +
        coord_flip() +
        scale_fill_brewer(palette = 'Set2') +
        theme_classic() +
        scale_y_continuous(labels = abs,
                           expand = c(0,0)) +
        facet_wrap(~Sex, 
                   strip.position = 'bottom', 
                   scale = 'free_x') +
        theme(text = element_text(size = 12), 
              panel.spacing.x = unit(0, 'pt'))

diab_data

In [ ]:
# Looking at binary survey data
diab_summary <- diab_data %>% 
        select(-Sex) %>%
        group_by(Diabetes_binary) %>% 
        summarize(across(where(~ is.factor(.x) && length(unique(.x)) == 2), ~ sum(. == "1") / n(), .names = '{.col}'))

survey_plot <- diab_summary %>%
        pivot_longer(!Diabetes_binary) %>%
        ggplot(aes(x = name, y = value, fill = Diabetes_binary)) +
        geom_bar(stat = 'identity', position = 'dodge', width = 0.7) +
        labs(x = '', y = 'Proportion answered "Yes"', fill = '') +
        ggtitle("Survey results across cases-controls") +
        scale_fill_brewer(palette = 'Set2') +
        theme_classic() +
        theme(axis.text.x = element_text(angle = 45, hjust = 1))

survey_plot

### 4. Methods :
These data encompass 21 variables, both categorical and numerical. We plan to identify 5 to 10 key variables that possess the highest predictive potential for diabetes, which we will employ to train a robust classifier. We will perform a short literature search, coupled with some basic data exploration techniques plus forward selection, to identify the variables that are most important in determining an individual’s risk for diabetes and overall guide our feature selection process. Once we have selected our features, we will train and test a variety of different predictive models, including K-nearest neighbors, random forests, and/or generalized linear models, to predict disease presence in a withheld test set. We will also use visualization tools such as PCA, PCoA, and/or T-SNE to represent these multidimensional data.

### 5. Expected outcomes and significance :
